In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
import datetime

In [ ]:
current_directory = os.getcwd()
print(current_directory)

In [ ]:
data = pd.read_csv('output.csv')

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [ ]:
# Find the minimum and maximum values of the "timestamp" column
min_timestamp = data['timestamp'].min()
max_timestamp = data['timestamp'].max()

# Print the results
print("Minimum Timestamp:", min_timestamp)
print("Maximum Timestamp:", max_timestamp)

In [ ]:
df_sorted_sentiment = data.sort_values('timestamp')
df_sorted_sentiment.head()

In [ ]:
df_daily_compound = df_sorted_sentiment.groupby(pd.Grouper(key='timestamp', freq='D')).agg({'compound_score': 'mean'}).reset_index()

In [ ]:
df_daily_compound['sentiment_signal'] = df_daily_compound['compound_score'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

In [ ]:
df_daily_compound.set_index('timestamp', inplace=True)
df_daily_compound.head(100)

In [ ]:
stockList = ["GME", "AMC"]

In [ ]:
startDate = "2021-01-28"
# startDate = "2020-10-30" # Adding 90 days befoe to have 90 days for the momentum strategy with 90 day period
# startDate = "2020-09-29"
endDate = "2021-08-16"

# startDate = "2020-01-01"
# endDate = "2021-12-31"

In [ ]:
df = yf.download(stockList, start=startDate, end=endDate)

In [ ]:
df.head()

In [ ]:
df['sentiment_signal'] = df_daily_compound['sentiment_signal']

In [ ]:
#Visualizing

plt.figure(figsize = (14,10))

#Line plots of stock prices over time. Note: this is actual price over that time period, which has been downloaded from Yahoo Finance

plt.subplot(2,2,1)
for symbol in stockList:
  plt.plot(df['Close'][symbol], label=f'{symbol} Close Price')

plt.title("Stock prices over time")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()

### Actual Returns of the 2 stocks

In [ ]:
actual_returns = df["Close"].iloc[-1]/df["Close"].iloc[0] - 1

print("Actual Returns:")

for symbol, return_value in actual_returns.items():

  print(f"{symbol}: {return_value:.2%}")

In [ ]:
# Create a bar plot for actual returns
plt.figure(figsize=(8, 6))
sns.barplot(x=actual_returns.index, y=actual_returns.values, palette='Set1')
plt.title('Actual Returns for GME and AMC')
plt.ylabel('Returns')
plt.xlabel('Stock Symbol')
plt.xticks(ticks=np.arange(len(actual_returns)), labels=actual_returns.index)
plt.tight_layout()
plt.show()

## Price Momentum Strategy:

The price momentum strategy is a type of trading strategy that capitalizes on the observed tendency of assets to continue moving in the same direction as their recent price trends. In other words, the strategy assumes that stocks that have recently shown strong positive price movements will continue to rise, and stocks that have recently exhibited weak price movements will continue to decline.

Step 1: Fetching historical data for these two stocks has been completed

Step 2: Calculating Momentum: The goal is to calculate the price momentum as the percentage change in the stock;s closing price over specified period (e.g.: 3 months):

In [ ]:
momentum_period = 90
for symbol in stockList:
    df['Momentum', symbol] = df['Close', symbol].pct_change(momentum_period)


In [ ]:
# Defining the moving average period
sma_period = 5  # Example: 10-day moving average. Adjust this based on your preferences.

thresholds = {}

for symbol in stockList:
    # Calculate the moving average of the momentum
    df['Smoothed_Momentum', symbol] = df['Momentum', symbol].rolling(window=sma_period).mean()

    # Set the buy and sell thresholds for the specific symbol
    buy_threshold = df['Smoothed_Momentum', symbol].quantile(0.75)
    sell_threshold = df['Smoothed_Momentum', symbol].quantile(0.25)
    thresholds[symbol] = {'buy': buy_threshold, 'sell': sell_threshold}

    # Generate signals based on the smoothed momentum and the respective thresholds
    df['Signal', symbol] = 0
    df.loc[df['Smoothed_Momentum', symbol] > thresholds[symbol]['buy'], ('Signal', symbol)] = 1
    df.loc[df['Smoothed_Momentum', symbol] < thresholds[symbol]['sell'], ('Signal', symbol)] = -1


In [ ]:
momentum_period = 90
buy_threshold_AMC = df['Smoothed_Momentum', 'AMC'].quantile(0.75)
sell_threshold_AMC = df['Smoothed_Momentum', 'AMC'].quantile(0.25)

buy_threshold_GME = df['Smoothed_Momentum', 'GME'].quantile(0.75)
sell_threshold_GME = df['Smoothed_Momentum', 'GME'].quantile(0.25)

# Initialize signals as 0
df[('Signal', 'AMC')] = 0
df[('Signal', 'GME')] = 0

# Iterate over rows to generate signals
for i, row in df.iterrows():
    # For AMC
    if row['Smoothed_Momentum', 'AMC'] > buy_threshold_AMC:
        df.at[i, ('Signal', 'AMC')] = 1
    elif row['Smoothed_Momentum', 'AMC'] < sell_threshold_AMC:
        df.at[i, ('Signal', 'AMC')] = -1

    # For GME
    if row['Smoothed_Momentum', 'GME'] > buy_threshold_GME:
        df.at[i, ('Signal', 'GME')] = 1
    elif row['Smoothed_Momentum', 'GME'] < sell_threshold_GME:
        df.at[i, ('Signal', 'GME')] = -1

# Implement Price Momentum Trading logic
capital = 100000
position = {symbol: 0 for symbol in stockList}
shares_owned = {symbol: 0 for symbol in stockList}

for i in range(1, len(df)):
    for symbol in stockList:
        momentum_signal = df.iloc[i][('Signal', symbol)]
        close_price = df.iloc[i][('Close', symbol)]
        
        # Check for buy signal
        if momentum_signal == 1 and position[symbol] == 0:
            shares_to_buy = capital // (2 * close_price)  # assuming equal distribution of capital
            shares_owned[symbol] += shares_to_buy
            capital -= shares_to_buy * close_price
            position[symbol] = 1
        
        # Check for sell signal
        elif momentum_signal == -1 and position[symbol] == 0:
            shares_to_sell_short = capital // (2 * close_price)
            shares_owned[symbol] -= shares_to_sell_short
            capital += shares_to_sell_short * close_price
            position[symbol] = -1
        
        # Check for neutral signal or closing positions
        elif momentum_signal == 0:
            if position[symbol] == 1:
                capital += shares_owned[symbol] * close_price
                shares_owned[symbol] = 0
            elif position[symbol] == -1:
                capital -= shares_owned[symbol] * close_price
                shares_owned[symbol] = 0
            position[symbol] = 0


# Calculate strategy returns
for symbol in stockList:
    df['Strategy_Returns', symbol] = df['Signal', symbol].shift(1) * df['Close', symbol].pct_change()
    df['Cumulative_Strategy_Returns', symbol] = (1 + df['Strategy_Returns', symbol]).cumprod()



In [ ]:
plt.figure(figsize=(14, 6))

# Plotting the momentum and smoothed momentum for AMC
plt.subplot(1, 2, 1)
plt.plot(df.index, df['Momentum', 'AMC'], label='AMC Momentum', color='blue')
plt.plot(df.index, df['Smoothed_Momentum', 'AMC'], label='AMC Smoothed Momentum', color='orange', linestyle='--')
plt.axhline(y=buy_threshold_AMC, color='g', linestyle='--', label=f'Buy Threshold (AMC: {buy_threshold_AMC:.2f})')
plt.axhline(y=sell_threshold_AMC, color='r', linestyle='--', label=f'Sell Threshold (AMC: {sell_threshold_AMC:.2f})')
plt.title('AMC Price Momentum over Time')
plt.xlabel('Date')
plt.ylabel('Momentum')
plt.legend()
plt.grid(True)

# Plotting the momentum and smoothed momentum for GME
plt.subplot(1, 2, 2)
plt.plot(df.index, df['Momentum', 'GME'], label='GME Momentum', color='red')
plt.plot(df.index, df['Smoothed_Momentum', 'GME'], label='GME Smoothed Momentum', color='purple', linestyle='--')
plt.axhline(y=buy_threshold_GME, color='g', linestyle='--', label=f'Buy Threshold (GME: {buy_threshold_GME:.2f})')
plt.axhline(y=sell_threshold_GME, color='r', linestyle='--', label=f'Sell Threshold (GME: {sell_threshold_GME:.2f})')
plt.title('GME Price Momentum over Time')
plt.xlabel('Date')
plt.ylabel('Momentum')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate the actual cumulative returns for GME and AMC for comparison
df['Cumulative_Return_GME'] = (1 + df['Close', 'GME'].pct_change()).cumprod()
df['Cumulative_Return_AMC'] = (1 + df['Close', 'AMC'].pct_change()).cumprod()

plt.figure(figsize=(14, 12))

# Plot for GME
plt.subplot(2, 2, 1)  
plt.plot(df['Cumulative_Return_GME'], label='Actual Returns', color='blue')
plt.plot(df['Cumulative_Strategy_Returns', 'GME'], label='Momentum Strategy Returns', color='green', linestyle='--')
plt.title('GME Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()

# Buy and Sell signals for GME
plt.subplot(2, 2, 2) 
plt.plot(df['Close', 'GME'], label='Price', color='blue')
plt.scatter(df[df['Signal', 'GME'] == 1].index, df['Close', 'GME'][df['Signal', 'GME'] == 1], marker='^', color='g', label='Buy Signal', alpha=1)
plt.scatter(df[df['Signal', 'GME'] == -1].index, df['Close', 'GME'][df['Signal', 'GME'] == -1], marker='v', color='r', label='Sell Signal', alpha=1)
plt.title('GME Buy & Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

# Plot for AMC
plt.subplot(2, 2, 3)  
plt.plot(df['Cumulative_Return_AMC'], label='Actual Returns', color='black')
plt.plot(df['Cumulative_Strategy_Returns', 'AMC'], label='Momentum Strategy Returns', color='orange', linestyle='--')
plt.title('AMC Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()

# Buy and Sell signals for AMC
plt.subplot(2, 2, 4) 
plt.plot(df['Close', 'AMC'], label='Price', color='black')
plt.scatter(df[df['Signal', 'AMC'] == 1].index, df['Close', 'AMC'][df['Signal', 'AMC'] == 1], marker='^', color='g', label='Buy Signal', alpha=1)
plt.scatter(df[df['Signal', 'AMC'] == -1].index, df['Close', 'AMC'][df['Signal', 'AMC'] == -1], marker='v', color='r', label='Sell Signal', alpha=1)
plt.title('AMC Buy & Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
df.head()

## Volatility Strategy

In [ ]:
# Bollinger Bands parameters
n = 20  # Number of days for SMA
k = 2  # Multiplier for standard deviation

for symbol in stockList:
    # Calculate the SMA for the Bollinger Bands
    df['SMA', symbol] = df['Close', symbol].rolling(window=n).mean()
    
    # Calculate the standard deviation for the Bollinger Bands
    df['STD', symbol] = df['Close', symbol].rolling(window=n).std()
    
    # Calculate the upper and lower Bollinger Bands
    df['Upper_Band', symbol] = df['SMA', symbol] + (df['STD', symbol] * k)
    df['Lower_Band', symbol] = df['SMA', symbol] - (df['STD', symbol] * k)
    
    # Generate volatility-based signals (Bollinger Bands)
    df['Volatility_Signal', symbol] = 0  # Default to 0
    df.loc[(df['Close', symbol].shift(1) < df['Lower_Band', symbol].shift(1)) & 
           (df['Close', symbol] > df['Lower_Band', symbol]), ('Volatility_Signal', symbol)] = 1
    df.loc[(df['Close', symbol].shift(1) > df['Upper_Band', symbol].shift(1)) & 
           (df['Close', symbol] < df['Upper_Band', symbol]), ('Volatility_Signal', symbol)] = -1


In [ ]:
def plot_stock_with_signals(df, symbol):
    plt.figure(figsize=(14, 7))

    # Plot closing price
    plt.plot(df.index, df['Close', symbol], label='Close Price', color='blue')
    
    # Plot Bollinger Bands
    plt.plot(df.index, df['Upper_Band', symbol], label='Upper Bollinger Band', color='lightgray')
    plt.plot(df.index, df['Lower_Band', symbol], label='Lower Bollinger Band', color='lightgray')
    
    # Get indices for buy and sell signals
    buy_signals = df[df['Volatility_Signal', symbol] == 1].index
    sell_signals = df[df['Volatility_Signal', symbol] == -1].index
    
    # Plot buy signals
    plt.scatter(buy_signals, 
                df.loc[buy_signals, ('Close', symbol)], 
                label='Buy Signal', marker='^', alpha=1, color='g', s=100)
    
    # Plot sell signals
    plt.scatter(sell_signals, 
                df.loc[sell_signals, ('Close', symbol)], 
                label='Sell Signal', marker='v', alpha=1, color='r', s=100)
    
    plt.title(f'{symbol} Close Price with Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot for AMC
plot_stock_with_signals(df, 'AMC')

# Plot for GME
plot_stock_with_signals(df, 'GME')


In [ ]:
import matplotlib.pyplot as plt

def plot_sentiment_signals(df, symbol):
    plt.figure(figsize=(15, 7))
    
    # Plotting the closing prices
    plt.plot(df.index, df['Close', symbol], label=f'{symbol} Close Price', alpha=0.7)
    
    # Plotting the buy signals
    buys = df[df['sentiment_signal'] == 1]
    plt.scatter(buys.index, df['Close', symbol][buys.index], label='Buy Signal', marker='^', color='g', alpha=1.0)
    
    # Plotting the sell signals
    sells = df[df['sentiment_signal'] == -1]
    plt.scatter(sells.index, df['Close', symbol][sells.index], label='Sell Signal', marker='v', color='r', alpha=1.0)
    
    plt.title(f'{symbol} Closing Price with Sentiment Buy/Sell Signals')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend(loc='best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Plot for AMC
plot_sentiment_signals(df, 'AMC')

# Plot for GME
plot_sentiment_signals(df, 'GME')


In [ ]:
import matplotlib.pyplot as plt

# Count the occurrences of each unique value
sentiment_counts = df['sentiment_signal'].value_counts()

# Generate the bar plot
sentiment_counts.plot(kind='bar', figsize=(10, 6), color=['skyblue', 'salmon', 'lightgreen'], alpha=0.8)

# Setting title and labels
plt.title('Distribution of Sentiment Signals')
plt.xlabel('Sentiment Signal')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y')

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
momentum_period = 90

# Define buy and sell thresholds for AMC and GME
buy_threshold_AMC = df['Smoothed_Momentum', 'AMC'].quantile(0.75)
sell_threshold_AMC = df['Smoothed_Momentum', 'AMC'].quantile(0.25)

buy_threshold_GME = df['Smoothed_Momentum', 'GME'].quantile(0.75)
sell_threshold_GME = df['Smoothed_Momentum', 'GME'].quantile(0.25)

# Initialize signals as 0
df[('Signal', 'AMC')] = 0
df[('Signal', 'GME')] = 0

# Iterate over rows to generate signals, incorporating sentiment
for i, row in df.iterrows():
    # For AMC
    if row['Smoothed_Momentum', 'AMC'] > buy_threshold_AMC and row['sentiment_signal'] == 1:
        df.at[i, ('Signal', 'AMC')] = 1
    elif row['Smoothed_Momentum', 'AMC'] < sell_threshold_AMC and row['sentiment_signal'] == -1:
        df.at[i, ('Signal', 'AMC')] = -1

    # For GME
    if row['Smoothed_Momentum', 'GME'] > buy_threshold_GME and row['sentiment_signal'] == 1:
        df.at[i, ('Signal', 'GME')] = 1
    elif row['Smoothed_Momentum', 'GME'] < sell_threshold_GME and row['sentiment_signal'] == -1:
        df.at[i, ('Signal', 'GME')] = -1
